# Chapter 10 - RANSAC & Bundle Adjustment

In this chapter we are looking into how we can construct a robust SFM. Since when working with SFM it can often happen that features are wrongly matched. If we determine the motion from the wrongly matched features we get totally wrong solution therefore we have to filter out such outliers first.



### Robust Estimation

When matching features there are usually some outliers, meaning point correspondences which are matched wrongly. Such outliers are caused by changing view points and changes is illumination. Also image noise, occlisions or blur can cause outliers. 

![Matching Example](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/1_matching_example.png)
*Figure 1: Matching Example. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

When working with Motion estimation such faulty estimation can cause our estimation to drift far away from the truth since all these small deviation do add up.

![Outliers on Motion Estimation](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/2_outliers_on_motion_estimation.png)
*Figure 2: Outliers on Motion Estimation. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

### RANdom SAmple Consensus (RANSAC)

A solution to having many outliers is the RANSAC method which is the standard method of fitting models in presence of having outliers. Since we have a normal fitting problem this solution can be applied whenever the goal is to estimate the parameters of a model. Therefore this is not exclusive to motion estimation and can allso be applied for example to SFM, Calibration or PnP and more.

The basic concept of RANSAC is that it samples randomly points and the tries to fit a model that describe these points. Then for all remaining points the error is calculated and the number of points inside e current region around our model/line are counted. Then the process is repeated. In the end we hselect the model with most points inside a certain region as the model.

So here is a example in more detail. Let go through the images from left to right. Given a set of points we select two points randomly. Then We fit a model throught these points. In this example we just draw a line. Then we calculate the error of all other points to that model. In our case this is just the shortest distance from our model/line to the point. Then we select all the data that support the hypothsis of our current model. This means we select the points that are inside a certain threshold around the model.

![Sampling and Fitting](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/3_random_sampling.png)
*Figure 3: Sampling and Fitting. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

This process is denn repeated for many different random samples.

![Repeating](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/4_repeating_process.png)
*Figure 4: Repeating. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

After k iterations we select the modle that hast most datapoint inside its support region. 

So how can we make sure that the produced results are reliable. As seen in the example when we only iterate 3 times and each time only select 2 points for fitting the model we will no reach a meaning full result. In the example above only at the forth iteration we reach a reasonable model. So what what is the optimal amount of point to use for fitting and what it the optmal number of iterations? 
So idelly we would always select 2 points for fitting and check all possible combinations of them. This however is computationallya very expensive since then there are $N(N-1)/2$ combinations possible for $N$ points. Even with only 1000 points this leadt to 500'000 combinations. So we would like to reduce the cost by not checking all possibilites and stop the RANSAC before. We can find out the optimal number of iterations in a probabilistic way.

#### Optimal k

First we have to define our prefered support range for this we have to set the inlierratio, so what ratio of all points have to be inside the support range. 

\begin{align*}
\textrm{inlier ratio} &= \textbf{w} := \textrm{number of inliers}/N \\
\textrm{number of points} &= \textbf{N}\\
\textrm{probaility of selecting inliers for both points} &= \mathbf{w^2} := P(p_1 == \textrm{inlier} \land p_2 == \textrm{inlier})\\
\textrm{probaility of selecting at least one outlier} &= \mathbf{1-w^2} := P(p_1 == \textrm{outlier} \lor p_2 == \textrm{outlier})\\
\textrm{RANSAC iteration} &= \textbf{k}\\
\textrm{probability that RANSAC never selcted both points as inlier} &= \mathbf{(1-w^2)^k} := (P(p_1 == \textrm{outlier} \lor p_2 == \textrm{outlier}))^2\\
\textrm{probaility of success} &= \textbf{p} := P(\textrm{both points inlier and therefore model reasonable})\\
\textrm{probaility of failure} &= \mathbf{1-p} := P(\textrm{never both points intlier, none of the models reasonable})
\end{align*}

\begin{align*}
&\implies 1-p = (1-w^2)^k\\
&\implies k = \dfrac{\log(1-p)}{\log(1-w^2)}\\
\end{align*}

So if we know the ratio of inliers $w$ and the probaility of success $p$ we can determine the number of iterations. Usually the user can choose these two parameters and therefor can decide the probaility of success and the inlier ratio. For example if we set the probaility of success ($p$) to 99% and the inlier ration $w$ to 50% we get $k=16$ iterations. This are significanlty fewer iterations than all combinations. Also notices that this derivation does not directly depend on the total number of points. Only $w$ depends on the total number of points.

Now we have derived a formula for getting the optimal number of iteration when we sample two points in each iteration. How does this change we we sample $s$ points in each iteration? Well the only thing that changes it that we don't have to take $w$ to the power of two but rather power $s$.

\begin{align*}
&\implies k = \dfrac{\log(1-p)}{\log(1-w^\mathbf{s})}
\end{align*}